<a href="https://colab.research.google.com/github/NneomaOkafor/HousingCAD/blob/main/Y_Data_Profile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check version

In [ ]:
!python --version


Python 3.11.13


Set up licence to use Ydata profiling

In [ ]:
import os
os.environ['YDATA_LICENSE_KEY'] = '{53505d93-3843-4f9c-bfbb-6c29a9ab6b8f}'

Install Y data profiling

In [ ]:
!pip install -q ydata-sdk ydata-profiling


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.4/408.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Set the file path
csv_path = '/content/drive/MyDrive/Data Analysis/FINAL PROJECT/Python files/Project_set_in_use.csv'


# 3 Read the CSV using YData's LocalConnector
from ydata.connectors import LocalConnector
from ydata.dataset.filetype import FileType

connector = LocalConnector()


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport

# Load your data
df = pd.read_csv(csv_path)

# Generate profile report (standard EDA mode)
profile = ProfileReport(df, title="Pandas Profiling Report")



In [ ]:
df = df.dropna(axis=1, how='all')  # drop columns with all NaNs

from ydata_profiling import ProfileReport

# Convert date column
df["Date"] = pd.to_datetime(df["Date"], errors='coerce')

# Run a lean but time-series-aware report
profile = ProfileReport(
    df,
    title="Time-Series EDA",
    tsmode=True,
    sortby="Date",
    explorative=False,  # saves time
    minimal=True        # skips heavy visualizations
)

#profile.to_file("/content/report_timeseries.html")


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error


Standardize the variables

In [ ]:
# Load your data
df = pd.read_csv ('/content/drive/MyDrive/Data Analysis/FINAL PROJECT/Python files/Project_set_in_use.csv')

# Ensure Date is datetime, sort chronologically
df["Date"] = pd.to_datetime(df["Date"])
df = df.sort_values("Date").reset_index(drop=True)

# Identify predictor columns (exclude target + Date)
predictor_cols = df.columns.difference(["Date", "Aggregate Composite MLS HPI"])

# Separate X, y
X_raw = df[predictor_cols].copy()
y     = df["Aggregate Composite MLS HPI"].values


Generate 6 months lag for every predictor; drop "average price" due to over dominance in variable selection

In [ ]:
# Ensure Date is in datetime format
df["Date"] = pd.to_datetime(df["Date"])

# Define predictor columns (excluding date and target)
predictor_cols = df.columns.difference(["Date", "Aggregate Composite MLS HPI"])

# Drop 'Average_Price' and its lags from predictors
X_raw = df[predictor_cols].drop(columns=[col for col in predictor_cols if "Average_Price" in col])

# Create lagged predictors
MAX_LAG = 6
def make_lags(df_in, max_lag):
    out = df_in.copy()
    for lag in range(1, max_lag + 1):
        out = out.join(df_in.shift(lag).add_suffix(f"_lag{lag}"))
    return out

X_lagged = make_lags(X_raw, MAX_LAG)

# After lag creation and cleaning
X_lagged["Aggregate Composite MLS HPI"] = df["Aggregate Composite MLS HPI"]
X_lagged = X_lagged.dropna()

# Separate clean X and y (keep y as Series to preserve index)
y = X_lagged["Aggregate Composite MLS HPI"]
X_lagged = X_lagged.drop(columns=["Aggregate Composite MLS HPI"])




Variable selection using the elastic method

In [ ]:
# Standardize all columns (they're already numeric)
scaler = StandardScaler()

# Elastic‑net cross‑validated: explore 50 λ values on a time‑series split
tscv = TimeSeriesSplit(n_splits=5)

enet_cv = ElasticNetCV(
    l1_ratio=[0.1, 0.25, 0.5, 0.75, 0.9],   # α grid: 0 = ridge … 1 = lasso
    cv=tscv,
    n_alphas=50,
    max_iter=10000,
    random_state=42,
    fit_intercept=True,
)

# Pipeline: Standardize → Elastic Net CV
pipeline = Pipeline([
    ("scale", StandardScaler()),
    ("enet", ElasticNetCV(
        l1_ratio=[0.1, 0.5, 0.9],
        n_alphas=50,
        max_iter=10000,
        cv=TimeSeriesSplit(n_splits=5),
        random_state=42
    ))
])

pipeline.fit(X_lagged, y)

Pipeline(steps=[('scale', StandardScaler()),
                ('enet',
                 ElasticNetCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                              l1_ratio=[0.1, 0.5, 0.9], max_iter=10000,
                              n_alphas=50, random_state=42))])

Fit and extract selected variables

In [ ]:
pipeline.fit(X_lagged, y)

# Best α (l1_ratio) and λ chosen by CV
best_alpha   = pipeline.named_steps["enet"].l1_ratio_
best_lambda  = pipeline.named_steps["enet"].alpha_
print(f"Best α (l1_ratio): {best_alpha},  Best λ: {best_lambda:.4f}")

# Coefficients
coefs = pipeline.named_steps["enet"].coef_
selected = pd.Series(coefs, index=X_lagged.columns)
selected_nonzero = selected[selected != 0].sort_values(key=np.abs, ascending=False)

print(f"\nSelected {selected_nonzero.shape[0]} variables out of {len(coefs)}:")
display(selected_nonzero.head(20))


Best α (l1_ratio): 0.9,  Best λ: 75.7002

Selected 2672 variables out of 2870:


,0
CLAIMS_QC,-1653.157341
CLAIMS_QC_lag1,-1597.336358
CLAIMS_QC_lag4,-1589.197580
CLAIMS_QC_lag3,-1584.016334
CLAIMS_QC_lag2,-1571.424316
CLAIMS_QC_lag5,-1567.065866
CLAIMS_QC_lag6,-1554.250675
CLAIMS_ALB_lag6,1450.047830
CLAIMS_ALB_lag5,1360.145761
N_DUR_INV_RAT_new_lag6,1289.870646
